# Designing a FIR filter with a window

In this notebook we are going to look at how to design a FIR filter using a window.
Do the imports

In [ ]:
%matplotlib widget
import scipy.signal as signal
import matplotlib.pyplot as plt
import numpy as np
np.seterr(divide='ignore')

def plotConv(fignum, title, x, h, fs):
    '''
    Function to plot the filter and signal

    Parameters
    ----------
    fignum : int
            figure number

    title : str
            The figure title

    x : numpy array
        The input signal

    h : numpy array
        The filter coefficients

    fs : float
        The sampling frequency input signal

    '''

    Nx = len(x)
    n = np.arange(Nx)

    
    plt.figure(fignum,figsize=(14,9))
    plt.clf()
    fig, ax = plt.subplots( nrows=2, ncols=3, figsize = (14, 9), num = fignum)


    # Plot the resulting input signal in the time domain
    ax[0, 0].plot(n, x, '-o')
    ax[0, 0].set_xlabel('$n$')
    ax[0, 0].set_ylabel('$x[n]$')
    ax[0, 0].set_title('Input signal')

    # Take the FFT of the input signal and plot it
    ax[1, 0].plot(n*fs/Nx, np.abs(np.fft.fft(x)), '-o')
    ax[1, 0].set_xlabel('$f$ (Hz)')
    ax[1, 0].set_ylabel('$X[f]$')
    ax[1, 0].set_title('FFT of input signal')

    # Take the FFT of the filter coefficients
    Nk = len(h)  # For holding the length of H

    H = np.fft.fft(h)
    k = np.arange(Nk)
    

    ax[0, 1].plot(k,h, '-o')
    ax[0, 1].set_xlabel('$h$')
    ax[0, 1].set_ylabel('$h[k]$ ')
    ax[0, 1].set_title('Filter coefficients')

    ax[1, 1].plot(k*fs/Nk, 20*np.log10(np.abs(H)), '-o')
    ax[1, 1].set_xlabel('$f$ (Hz)')
    ax[1, 1].set_ylabel('$H[f]$ (dB)')
    ax[1, 1].set_title('Frequency respons (FFT) of the filter')


    # Plot the phase response
    #ax[1, 1].plot(k*fs/Nk, np.arctan2(np.imag(H), np.real(H)), 'o')
    #ax[1, 1].set_xlabel('$f$ (Hz)')
    #ax[1, 1].set_ylabel('$H_{\phi}[f]$ (rad)')
    #ax[1, 1].set_title('Phase  response (FFT) of the filter')

    # Apply the filter to the input signal using convolution
    y = np.convolve(h, x)

    Ny = len(y)
    ny = np.arange(Ny)

    # Plot the resulting output signal
    ax[0, 2].plot(ny, y, '-o')
    ax[0, 2].set_xlabel('$n$')
    ax[0, 2].set_ylabel('$y[n]$')
    ax[0, 2].set_title('Output signal')

    # Plot the FFT of the signal
    ax[1, 2].plot(ny*fs/Ny, np.abs(np.fft.fft(y)), '-o')
    ax[1, 2].set_xlabel('$f$ (Hz)')
    ax[1, 2].set_ylabel('$Y_{mag}$')
    ax[1, 2].set_title('FFT of Output signal')
    plt.show()



Design the window, here a square window for $H(m)$

In [ ]:
Nh = 64

# Initialising  an array to hold H
H = np.zeros(Nh)

# Set the one sided length (in m) of the filter
K2 = 5  # K/2+1

# Write this on both sides of the H (remembering the symmetry). The DC-component
# (m=0) is not written.
H[0:K2+1] = 1
H[Nh-K2:Nh] = 1


# Plot the resulting frequency response as a function of m
plt.figure(1)
plt.clf()
plt.plot(H, 'o')
plt.xlabel('$m$')
plt.ylabel('$H[m]$')
plt.grid()
plt.tight_layout()
plt.show()


Notice that the x-axis is in $m$, and not in frequency, meaning that the cut-off
frequency will depend on the sampling frequency.

# Calculating the filter coefficients

We are now going to calculate the filter coefficients by doing the invers DFT
$$
h(k) = \frac{1}{N}\sum_{m = 0} ^ {N-1} H(m) e ^ {j 2\pi m k / N}
$$

In [ ]:
# Finding the filter coefficients by taking the inverse FFT
hr = np.fft.ifft(H, Nh)

hr = np.real(hr)  # We are only want the real values

# Plot the figure of the filter coefficients
plt.figure(2)
plt.clf()
plt.plot(hr, 'o')
plt.xlabel('$k$')
plt.ylabel('$h[k]$')
plt.tight_layout()
plt.show()

In the figure we need to check that the filter coefficients are symetrical and
don't wary to much.

In [ ]:
# We want to shift it to the center to make the final filter symmetric

h = np.roll(hr, Nh//2)  # Roll the filter 32 steps

h = h[1:]  # Removing the point making it non-symetric

plt.figure(3)
plt.clf()
plt.plot(h, 'o')
plt.xlabel('$k$')
plt.ylabel('$h[k]$')
plt.grid()
plt.tight_layout()
plt.show()


Once you are sattisfied about this we will make the signal to test how the filter
works.


In [ ]:
Nx = 100
nx = np.arange(Nx)

fs = 50  # Sampling frequency in Hz
ts = 1/fs  # Sampling time in s

f1 = 1  # Frequency of the first signal in Hz
f2 = 20  # Frequency of the second signal in Hz

# Making the signal as a sum of two cosines.
x = np.cos(2*np.pi*f1*nx*ts) + np.cos(2*np.pi*f2*nx*ts)


We can now visualise the signal, the filter and the output signal found as the convolution of the input signal $x[n]$ and the filter koefficients $h[k]$.

In [ ]:

plotConv(4, 'Rectangular LP filter', x, h, fs)


## Lowpass to bandpass 
Make a bandpass filter out of the LP with a center frequency of
$$ f_c = \frac{f_s}{4} $$


In [ ]:
fc = fs / 4 # The center frequency
ns = np.arange(len(h)) # The n values for the shift function, with the same length as h

Sskift = np.sin(2 * np.pi * fc * ns / fs) # The shift function

# Calculate the  resulting bandpass filter
hbp = Sskift * h

# Plotting it 
plotConv(5, 'Rectangular BP filter', x, hbp, fs)

## Low pass to highpass 
Make a bandpass filter out of the LP with a center frequency of
$$ f_c = \frac{f_s}{2} $$


In [ ]:
fc = fs / 2 # The center frequency

# We want a sequence that goes (1, -1, 1, -1 … ) 
Sskift = np.cos(2 * np.pi * fc * ns / fs) # The shift function

# Calculate the  resulting bandpass filter
hhp = Sskift * h

# Plotting it 
plotConv(6, 'Rectangular HP filter', x, hhp, fs)


# Manual window filter design 
In practise we would use windows and not rectanguar selections to generate our filter response. We are now going to do this. 

We will first take a look at the window and see how it looks. The window of choice is the Dolph-Chebyshev window.

In [ ]:

# Make the window, by setting the number of coefficients, M, and the attenu
M = 64  # Number of coefficients
at = 100  # Attenuation of the window in dB

# Get the window and plot it 
w = signal.get_window(('chebwin', at), M)

plt.figure(7)
plt.clf()
plt.plot(w, '.')
plt.title("Dolph-Chebyshev window (100 dB)")
plt.ylabel("$H[m]$")
plt.xlabel("$m$")
plt.show()


As can be seen, this is a highpass filter, and not a lowpass, meaning that we need to shift it

In [ ]:
# We want a lowpass 
ww = np.roll(w, M//2)

# Find the resulting real filter coefficients
htmp = np.real(np.fft.ifft(ww, M))

hw = np.roll(htmp, M//2)

# Remove the non-symetric 
hw = hw[1:]  

# Plot how the filter works on the signal

plotConv(8, 'Dolph-Chebyshev LP filter', x, hw, fs)


## Window FIR filter design using algorithms 

We will now use the *scipy* method of [firwin](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html)  


In [ ]:

# We now add a new signal to x in order to test how bandpass / bandstop filter might work 
f3 = 7 # Frequency in Hz

z= x + np.cos(2 * np.pi * f3 * nx / fs + np.pi / 5.4)

taps = 71 # Number of taps in the filter 
fcut = [5, 9] # Cut of frequencies in Hz
# Make the bandstop filter using firwin, check the documentation for explanation of parameters

hf = signal.firwin(taps, fcut, pass_zero=False, window=('chebwin', 100), fs=fs)

# Apply and plot the filter 
plotConv(9, 'FIRwin filter', z, hf, fs)